In [1]:
import pickle
from copy import deepcopy
import numpy as np

class data_object(object):
    def __init__(self, name):
        self.name = name

In [2]:
master_collector = {}

for i in range(1,1001):
    rand_input = data_object('{}'.format(i))
    '''
    model input, randomized
    '''
    # The following parameters requires manual input
    rand_input.tray_number = 20
    mpcc_type = 'NCP'
    rand_input.non_reactive_flag = [1,2,3,4,5,6,7]

    # The following parameters is generated manually
    # reset random seeds
    np.random.seed()

    # distillate ratio = 0.05 - 0.15 total condenser liquid
    rand = np.random.rand()
    rand_input.rr_ratio = 0.05 + 0.1*(rand)

    # generate three product flag, randint produces [low,high)
    rand_input.side_draw_flag = {}
    # intermediate 2 - 4
    rand_int = np.random.randint(low=2,high=5)
    rand = np.random.rand()
    rand_input.side_draw_flag.update({rand_int:0.01+0.03*rand})
    # gasoline 5 - 9
    rand_int = np.random.randint(low=5,high=10)
    rand = np.random.rand()
    rand_input.side_draw_flag.update({rand_int:0.1+0.2*rand})
    # diesel 11 - 18
    rand_int = np.random.randint(low=11,high=19)
    rand = np.random.rand()
    rand_input.side_draw_flag.update({rand_int:0.2+0.3*rand})

    # generating temperature profile
    profile_tmp = sorted(np.random.rand(rand_input.tray_number - len(rand_input.non_reactive_flag)))
    rand_input.temperature_flag = {}; n = 0
    for j in range(1,rand_input.tray_number+1):
        if j not in rand_input.non_reactive_flag:
            rand_input.temperature_flag[j] = 220+60*profile_tmp[n]
            n += 1
    
    # generating feed profile, total = 10 kmol/s
    profile_tmp = {j:np.random.rand() for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    total_tmp = sum(profile_tmp[j] for j in profile_tmp)
    rand_input.feed_flag = {j:0.01 + (10-0.01*len(profile_tmp))*profile_tmp[j]/total_tmp for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    rand_input.feed_flag.update({j:0 for j in rand_input.non_reactive_flag})

    # generating catalyst profile, total = 30000 kg
    profile_tmp = {j:np.random.rand() for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    total_tmp = sum(profile_tmp[j] for j in profile_tmp)
    rand_input.catalyst_flag = {j:100 + (30000-100*len(profile_tmp))*profile_tmp[j]/total_tmp for j in range(1,rand_input.tray_number+1) if j not in rand_input.non_reactive_flag}
    rand_input.catalyst_flag.update({j:0 for j in rand_input.non_reactive_flag})
    
    master_collector[i] = rand_input

In [3]:
# with open('random_states_2.pickle','wb') as f:
#     pickle.dump(master_collector,f)